# streamlit

In [ ]:
!pip install streamlit smolagents sqlalchemy pandas
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹
changed 22 packages in 2s
⠹
⠹3 packages are looking for funding
⠹  run `npm fund` for details
⠹

In [ ]:
!wget -O manutencao_industrial.db "https://drive.google.com/uc?export=download&id=1B4l59bdPycqB4peRQxrrgOIlmGkGi-0v"

--2025-07-01 06:15:56--  https://drive.google.com/uc?export=download&id=1B4l59bdPycqB4peRQxrrgOIlmGkGi-0v
Resolving drive.google.com (drive.google.com)... 74.125.201.113, 74.125.201.101, 74.125.201.139, ...
Connecting to drive.google.com (drive.google.com)|74.125.201.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1B4l59bdPycqB4peRQxrrgOIlmGkGi-0v&export=download [following]
--2025-07-01 06:15:56--  https://drive.usercontent.google.com/download?id=1B4l59bdPycqB4peRQxrrgOIlmGkGi-0v&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 173.194.195.132, 2607:f8b0:4001:c11::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|173.194.195.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 24576 (24K) [application/octet-stream]
Saving to: ‘manutencao_industrial.db’

manutencao_industri 100%[===================>]  24.00K 

In [ ]:
%%writefile app.py
import streamlit as st
from sqlalchemy import create_engine, inspect, text
from smolagents import tool, CodeAgent, InferenceClientModel
import os

# Configuração token
os.environ["HF_TOKEN"] = ""

# Classe do agente
class IndustrialMaintenanceAgent:
    def __init__(self, db_path="/content/manutencao_industrial.db"):
        self.db_path = db_path
        self.engine = create_engine(f"sqlite:///{db_path}")
        self.setup_sql_tool()
        self.create_agent()

    def analyze_database_structure(self):
        """Analisa a estrutura do banco de dados e retorna uma descrição detalhada"""
        inspector = inspect(self.engine)
        tables = inspector.get_table_names()

        description = "Banco de dados de manutenção industrial. Tabelas disponíveis:\n\n"

        for table in tables:
            columns_info = inspector.get_columns(table)
            foreign_keys = inspector.get_foreign_keys(table)

            description += f"Tabela '{table}':\n"
            description += "Colunas:\n"

            for col in columns_info:
                col_desc = f" - {col['name']}: {col['type']}"
                if not col['nullable']:
                    col_desc += " (NOT NULL)"
                if col.get('primary_key'):
                    col_desc += " (CHAVE PRIMÁRIA)"
                description += col_desc + "\n"

            if foreign_keys:
                description += "Relações:\n"
                for fk in foreign_keys:
                    description += f" - {fk['constrained_columns']} → {fk['referred_table']}.{fk['referred_columns']}\n"

            description += "\n"

        return description

    def setup_sql_tool(self):
        """Configura a ferramenta SQL com a descrição do banco de dados"""
        db_description = self.analyze_database_structure()

        @tool
        def sql_maintenance_query(query: str) -> str:
            """
            Permite realizar consultas SQL no banco de dados de manutenção industrial.
            Retorna uma string com os resultados.

            O banco de dados contém informações sobre manutenção industrial incluindo:
            - Ordens de manutenção com datas e descrições
            - Técnicos com especialidades e turnos de trabalho
            - Equipamentos industriais com tipos e especificações
            - Relações entre ordens, técnicos e equipamentos

            Args:
                query: Consulta SQL a ser executada. Deve ser SQL válido.
            """
            try:
                # Obter descrição do banco para contexto adicional
                db_info = self.analyze_database_structure()

                with self.engine.connect() as con:
                    result = con.execute(text(query))
                    rows = result.fetchall()

                    if not rows:
                        return "Nenhum resultado encontrado."

                    # Formatar resultados de forma legível
                    columns = result.keys()
                    output = "Resultados:\n"
                    output += " | ".join(columns) + "\n"
                    output += "-" * (len(" | ".join(columns))) + "\n"

                    for row in rows:
                        output += " | ".join(str(value) for value in row) + "\n"

                    return output

            except Exception as e:
                return f"Erro na consulta SQL: {str(e)}"

        self.sql_tool = sql_maintenance_query

    def create_agent(self):
        """Cria o agente com o modelo especificado"""
        self.agent = CodeAgent(
            tools=[self.sql_tool],
            model=InferenceClientModel(#"microsoft/Phi-3.5-mini-instruct"
                #model_id="Qwen/Qwen2.5-Coder-32B-Instruct"
                model_id="Qwen/Qwen2-72B-Instruct"
            ),
            max_steps = 5,
        )

    def query(self, question: str):
        """Processa uma pergunta em linguagem natural"""
        try:
            response = self.agent.run(question)
            return response
        except Exception as e:
            return f"Erro ao processar a pergunta: {str(e)}"
# CSS personalizado para melhor aparência
st.set_page_config(
    page_title="Agente de Manutenção Industrial",
    layout="wide",
    page_icon="🏭"
)

st.markdown("""
<style>
    .main-header {
        font-size: 2.5rem;
        color: #1f77b4;
        text-align: center;
        margin-bottom: 2rem;
    }
    .chat-message {
        padding: 1rem;
        border-radius: 0.5rem;
        margin: 0.5rem 0;
    }
    .user-message {
        background-color: #e3f2fd;
        border-left: 4px solid #1976d2;
    }
    .bot-message {
        background-color: #f5f5f5;
        border-left: 4px solid #4caf50;
    }
    .sidebar {
        background-color: #f8f9fa;
    }
</style>
""", unsafe_allow_html=True)

# Instanciar o agente
@st.cache_resource
def load_agent():
    return IndustrialMaintenanceAgent()

try:
    agent = load_agent()
    agent_loaded = True
except Exception as e:
    st.error(f"Erro ao carregar o agente: {str(e)}")
    agent_loaded = False

# Interface do Streamlit
st.markdown('<h1 class="main-header">🏭 Agente Inteligente de Manutenção Industrial</h1>', unsafe_allow_html=True)

# Sidebar com informações
with st.sidebar:
    st.header("ℹ️ Informações")

    if agent_loaded:
        st.success("✅ Agente carregado com sucesso")

        with st.expander("📊 Estrutura do Banco"):
            st.text(agent.analyze_database_structure())
    else:
        st.error("❌ Erro ao carregar o agente")

    st.header("💡 Exemplos de Perguntas")
    exemplos = [
        "Qual técnico trabalhou na ordem 32?",
        "Quantas ordens de manutenção foram realizadas?",
        "Quais equipamentos precisam de manutenção?",
        "Qual técnico trabalhou em mais ordens?",
        "Em qual turno ocorrem mais manutenções?"
    ]

    for exemplo in exemplos:
        if st.button(f"📝 {exemplo}", key=f"ex_{hash(exemplo)}"):
            st.session_state.exemplo_selecionado = exemplo

# Inicializar histórico de mensagens
if "chat_history" not in st.session_state:
    st.session_state.chat_history = []

# Área principal de chat
col1, col2 = st.columns([3, 1])

with col1:
    st.subheader("💬 Faça sua pergunta")

    # Input do usuário
    user_input = st.text_input(
        "Digite sua pergunta :",
        placeholder="Ex: Qual técnico trabalhou na ordem 32?",
        key="user_input"
    )

    # Verificar se há exemplo selecionado
    if "exemplo_selecionado" in st.session_state:
        user_input = st.session_state.exemplo_selecionado
        del st.session_state.exemplo_selecionado

    # Processar pergunta
    if st.button("📤 Enviar Pergunta") and user_input and agent_loaded:
        with st.spinner("🔄 Processando sua pergunta..."):
            response = agent.query(user_input)
            print(f"RESPUESTA: {response}")
            # Adicionar ao histórico
            st.session_state.chat_history.append(("Você", user_input))
            st.session_state.chat_history.append(("Agente", response))

        # Limpar input
        st.rerun()

with col2:
    st.subheader("🗂️ Ações")

    if st.button("🗑️ Limpar Histórico"):
        st.session_state.chat_history = []
        st.rerun()

    if st.button("🔄 Recarregar Agente"):
        st.cache_resource.clear()
        st.rerun()

# Mostrar histórico de chat
st.subheader("📝 Histórico da Conversa")

if st.session_state.chat_history:
    # Mostrar mensagens do mais recente para o mais antigo
    for i, (sender, msg) in enumerate(reversed(st.session_state.chat_history)):
        if sender == "Você":
            st.markdown(f"""
            <div class="chat-message user-message">
                <strong>👤 {sender}:</strong><br>
                {msg}
            </div>
            """, unsafe_allow_html=True)
        else:
            st.markdown(f"""
            <div class="chat-message bot-message">
                <strong>🤖 {sender}:</strong><br>
                {msg}
            </div>
            """, unsafe_allow_html=True)
else:
    st.info("💡 Nenhuma conversa ainda. Faça sua primeira pergunta!")


# Executar Streamlit
print("🚀 Para executar:")
print("1. Salve este código como app.py")
print("2. Execute: streamlit run app.py")
print("3. Acesse http://localhost:8501")

Overwriting app.py


In [ ]:
import os
print("Archivos en el directorio:")
for file in os.listdir():
    print(f"- {file}")

# Verificaçao base de dados
import sqlite3
try:
    conn = sqlite3.connect('manutencao_industrial.db')
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = cursor.fetchall()
    print(f"\nTabelas na base de dados: {[table[0] for table in tables]}")
    conn.close()
    print("✅ Base de dados válida")
except Exception as e:
    print(f"❌ Error na base de dados: {e}")

Archivos en el directorio:
- .config
- manutencao_industrial.db
- package.json
- package-lock.json
- stremsli.py
- app.py
- logs.txt
- .ipynb_checkpoints
- node_modules
- sample_data

Tablas en la base de datos: ['equipamentos', 'ordens_manutencao', 'tecnicos', 'ordem_tecnico']
✅ Base de datos válida


In [ ]:
#  Streamlit con túnel
import subprocess
import threading
import time
import urllib.request

def run_streamlit():
    """Ejecuta Streamlit en background"""
    subprocess.run([
        "streamlit", "run", "app.py",
        "--server.port", "8501",
        "--server.headless", "true",
        "--server.fileWatcherType", "none",
        "--browser.gatherUsageStats", "false"
    ])

# Obter IP pública para localtunnel
def get_public_ip():
    try:
        return urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip()
    except:
        return "Unable to get IP"

# Iniciar Streamlit em background
print("🚀 Iniciando Streamlit...")
streamlit_thread = threading.Thread(target=run_streamlit)
streamlit_thread.daemon = True
streamlit_thread.start()

# Esperar inicialização Streamlit 
print("⏳ Esperando a que Streamlit inicie...")
time.sleep(15)

# Obter IP pública
public_ip = get_public_ip()
print(f"🌐 IP pública: {public_ip}")
print(f"📌 Password/Endpoint para localtunnel: {public_ip}")

# Iniciar túnel
print("🔗 Creando túnel con localtunnel...")
print("⚠️ IMPORTANTE: Use la IP mostrada arriba como password cuando se solicite")

🚀 Iniciando Streamlit...
⏳ Esperando a que Streamlit inicie...
🌐 IP pública: 34.172.111.157
📌 Password/Endpoint para localtunnel: 34.172.111.157
🔗 Creando túnel con localtunnel...
⚠️ IMPORTANTE: Use la IP mostrada arriba como password cuando se solicite


In [ ]:
!npx localtunnel --port 8501

⠙your url is: https://sour-banks-float.loca.lt
^C
